In [1]:
import tensorflow as tf
print("tensorflow: ", tf.__version__)

from tensorflow import keras
print("keras:", keras.__version__)

In [2]:
#-- NumPy is a Python library used for working with arrays.
import numpy as np 
#-- Pandas is a Python library used for working with data sets.
import pandas as pd 
#-- Pyplot contains various functions that help matplotlib behave like MATLAB.
import matplotlib
#-- Agg, is a non-interactive backend that can only write to files. 
#-- It is used on Linux, if Matplotlib cannot connect to either an X display.
matplotlib.use('agg')
import matplotlib.pyplot as plt 

#-- OpenCV-Python is a library of Python bindings designed to solve computer vision problems
import cv2
#-- Performs interpolation to up-size or down-size images.
from skimage.transform import resize 

#-- The OS module in python provides functions for interacting with the operating system.
import os
#-- pseudo-random generator
import random 
#-- regular expression
import re

#-- Switching from one backend to another.
from tensorflow.keras import backend as K 
#-- A collection of examples for learning mathematical modelling
from tensorflow.keras import optimizers 
#-- Keras ImageDataGenerator is used for getting the input of the original data and further, 
#--  it makes the transformation of this data on a random basis and gives the output
# from tensorflow.keras.preprocessing.image import ImageDataGenerator 
#-- collection of utilities for array and list manipulation
from tensorflow.keras.utils import to_categorical
#-- Sequential groups a linear stack of layers into a tf.keras.Model.  
#-- Sequential provides training and inference features on this model.
from tensorflow.keras.models import Sequential, load_model 
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout
#-- Loads an image into PIL format.
from tensorflow.keras.utils import load_img

In [3]:
train_dir = "../input/ai-mnist/MNIST/train/"
train_data = []

for i in os.listdir(train_dir):
    sub_directory = os.path.join(train_dir,i)
    count = 0
    for j in os.listdir(sub_directory):
        count+=1
        if count > 3000:
            break
        #-- Load the image, force it to be in grayscale format, and force the size to be 28×28 pixels
        filename = os.path.join(sub_directory,j)
        img = load_img(filename, color_mode="grayscale", target_size=(28,28))
        img = np.array(img)
        img = img.reshape(28,28,1)
        train_data.append([img,i])
len(train_data)

In [4]:
val_dir = "../input/ai-mnist/MNIST/validation/"
val_data = []

for i in os.listdir(val_dir):
    sub_directory = os.path.join(val_dir,i)
    count = 0
    for j in os.listdir(sub_directory):
        count+=1
        if count > 750:
            break
        #-- Load the image, force it to be in grayscale format, and force the size to be 28×28 pixels
        filename = os.path.join(sub_directory,j)
        img = load_img(filename, color_mode="grayscale", target_size=(28,28))
        img = np.array(img)
        img = img.reshape(28,28,1)
        val_data.append([img,i])
len(val_data)

In [5]:
random.shuffle(train_data)
random.shuffle(val_data)

train_X = []
_train_Y = []
for features,label in train_data:
    train_X.append(features)
    _train_Y.append(label)
    
val_X = []
_val_Y = []
for features,label in val_data:
    val_X.append(features)
    _val_Y.append(label)

In [6]:
#-- Inspect a few examples in the dataset
fig = plt.figure()
for i in range(9):
  plt.subplot(3,3,i+1)
  plt.tight_layout()
  plt.imshow(train_X[i], cmap='gray', interpolation='none')
  plt.title("Digit: {}".format(_train_Y[i]))
  plt.xticks([])
  plt.yticks([])
fig

In [7]:
#-- In order to train the neural network to classify images,
#--   unroll the height x width pixel format into one big vector (input vector). 
#-- The length must be 28 x 28 = 1024. 
#-- This is an example of the distribution graph of the pixel values.
fig = plt.figure()
j = 0
plt.subplot(2,1,1)
plt.imshow(train_X[j], cmap='gray', interpolation='none')
plt.title("Digit: {}".format(_train_Y[j]))
plt.xticks([])
plt.yticks([])
plt.subplot(2,1,2)
plt.hist(train_X[j].reshape(784))
plt.title("Pixel Value Distribution")
fig

In [8]:
#-- Pixel values range from 0 to 255: the background majority close to 0 (black), 
#--   and those close to 255 (white) representing the digit.
#-- Normalize the pixel values to lie between 0 and 1
train_X = np.array(train_X)
train_X = train_X.astype('float32')
train_X = train_X.reshape(train_X.shape[0],28,28,1)
train_X /= 255

val_X = np.array(val_X)
val_X = val_X.astype('float32')
val_X = val_X.reshape(val_X.shape[0],28,28,1)
val_X /= 255

_train_Y = np.array(_train_Y, dtype='uint8')
_val_Y = np.array(_val_Y, dtype='uint8')

#-- Print the final shape ready for training
print("matrix shape (train_X|_train_Y) :", train_X.shape,_train_Y.shape)
print("matrix shape (  val_X|  _val_Y) :", val_X.shape,_val_Y.shape)

In [9]:
#-- the truth (Y in machine learning lingo) used for training still holds integer values from 0 to 9.
#-- uint8 is an unsigned 8-bit integer that can represent values 0..255
print(np.unique(_val_Y, return_counts=True))

In [10]:
#-- one-hot encoding using keras' numpy-related utilities
#-- The vector is all zeroes except in the position for the respective category.
#-- For example, a '5' will be represented by [0,0,0,0,0,1,0,0,0,0].
n_classes = 10
print("Shape (_train_Y) before one-hot encoding:", _train_Y.shape)
print("Shape (  _val_Y) before one-hot encoding:", _val_Y.shape)
train_Y = keras.utils.to_categorical(_train_Y, n_classes)
val_Y = keras.utils.to_categorical(_val_Y, n_classes)
print("Shape (train_Y) after  one-hot encoding:", train_Y.shape)
print("Shape (  val_Y) after  one-hot encoding: ", val_Y.shape)

In [11]:
#-- Building a linear stack of layers with the sequential model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28,28,1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(10, activation='softmax'))

model.summary()

In [12]:
#-- Compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')
#model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer=SGD)

In [13]:
#-- Training the model and saving metrics in history
history = model.fit(train_X, train_Y,
          batch_size=5000, epochs=50,
          verbose=2,
          validation_data=(val_X, val_Y))

In [14]:
model.save('handwriting.h5')

In [15]:
#-- Plotting the metrics
fig = plt.figure()
plt.subplot(2,1,1)
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')

plt.subplot(2,1,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')

plt.tight_layout()

fig

In [16]:
model = load_model('handwriting.h5')
loss_and_metrics = model.evaluate(val_X, val_Y, verbose=2)

print("Validation Loss", loss_and_metrics[0])
print("Validation Accuracy", loss_and_metrics[1])

In [17]:
#-- Load the model and create predictions on the validation set
model = load_model('handwriting.h5')
predicted_classes = np.argmax(model.predict(val_X), axis=1)

#-- See which we predicted correctly and which not
correct_indices = np.nonzero(predicted_classes == _val_Y)[0]
incorrect_indices = np.nonzero(predicted_classes != _val_Y)[0]
print()
print(len(correct_indices)," classified correctly")
print(len(incorrect_indices)," classified incorrectly")

In [18]:
#-- Adapt figure size to accomodate 18 subplots
plt.rcParams['figure.figsize'] = (7,14)

figure_evaluation = plt.figure()

#-- Plot 9 correct predictions
for i, correct in enumerate(correct_indices[:9]):
    plt.subplot(6,3,i+1)
    plt.imshow(val_X[correct].reshape(28,28), cmap='gray', interpolation='none')
    plt.title(
      "Predicted: {}, Truth: {}".format(predicted_classes[correct],
                                        _val_Y[correct]))
    plt.xticks([])
    plt.yticks([])

#-- Plot 9 incorrect predictions
for i, incorrect in enumerate(incorrect_indices[:9]):
    plt.subplot(6,3,i+10)
    plt.imshow(val_X[incorrect].reshape(28,28), cmap='gray', interpolation='none')
    plt.title(
      "Predicted: {}, Truth: {}".format(predicted_classes[incorrect],
                                        _val_Y[incorrect]))
    plt.xticks([])
    plt.yticks([])

figure_evaluation

In [29]:
def prediction(filename):
    number = re.search(r"\d",filename)
    actual = number.group()
    mypath = os.path.join("../input/ai-mnist/MNIST/holdout_32x32",filename)
    img = load_img(mypath, color_mode="grayscale", target_size=(28,28))
    img = np.invert(img)
    img = img.astype('float32')
    img_re = img.reshape(28,28,1)
    img_re /= 255
    img_re = resize(img_re, (28, 28, 1))
    probabilities = model.predict(np.array([img_re],))[0,:]
    number_to_class = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    index = np.argsort(probabilities)
    if number_to_class[index[9]] == actual:
     grade = "Good Job!"
    else:
     grade = "Hmmm ... Did I make a wrong guess?"
    predictions = {
      "actual":actual,
      "digit":number_to_class[index[9]],
      "prob" :probabilities[index[9]],
      "comment":grade
    }
    return predictions, img_re

In [40]:
#-- Correct
predictions = ()
predictions, img_re = prediction("0.jpg")
#predictions, img_re = prediction("1.jpg")
#predictions, img_re = prediction("2.jpg")
#predictions, img_re = prediction("3.jpg")
#predictions, img_re = prediction("4.jpg")
#predictions, img_re = prediction("5.jpg")
#predictions, img_re = prediction("6.jpg")
#predictions, img_re = prediction("7.jpg")
#predictions, img_re = prediction("8.jpg")
#predictions, img_re = prediction("9.jpg")

print(predictions)
plt.imshow(img_re, cmap='gray', interpolation='none')

In [21]:
#import shutil
#shutil.rmtree("/kaggle/working/")